In [220]:
import pandas as pd
import numpy as np

In [221]:
data = pd.read_json('amenities-vancouver.json.gz', lines = True, encoding="utf8")

In [222]:
data = data.drop('timestamp', axis = 1)
# data = data['amenity'].drop_duplicates()

In [223]:
print(data['lat'].min())
print(data['lat'].max())
print(data['lon'].min())
print(data['lon'].max())

49.0012885
49.4633008
-123.4998587
-122.0006559


In [224]:
# data.groupby('amenity').count()
# to_csv('agg.csv')

In [6]:
If I was planning a tour of the city (by walking/biking/driving), where should I go? 
Are there paths that take me past an interesting variety of things?
We can break down recommendations based on the different forms of transportation (Nhi)
Adding to the previous question, what certain times of the day are entertainment amenities really busy 
and also group them by geographic location. (Nhi)


SyntaxError: invalid syntax (<ipython-input-6-008b83226efb>, line 1)

In [225]:
food = ['cafe', 'fast_food', 'restaurant','pub','bistro','bar','ice_cream',
        'biergarten','food_court']
drink = ['juice_bar', 'internet_cafe']
place = ['public_building',  'clock', 'marketplace',
         'conference_centre','Observation Platform','watering_place',
         'arts_centre','park', 'casino', 'shop|clothes','townhall' ]
car = ['car_rental','charging_station', 'car_sharing','fuel','EVSE']
parking = ['parking_entrance','parking','parking_space']
walk = ['taxi', 'bus_station']
bike = ['bicycle_parking','bicycle_rental']
boat = ['boat_rental']
moto = ['motorcycle_rental','motorcycle_parking']

In [226]:
interesting = food+drink+place+car+parking+walk+bike+boat+moto
intPlace = data[data['amenity'].isin(interesting)]

In [227]:
print(data.shape)
print(intPlace.shape)
print(len(interesting))

(17718, 5)
(7890, 5)
37


In [93]:
# TODO: distribution of those places, how many of each

In [94]:
# clean data with only 'interesting places'
# intPlace.groupby('amenity').count()

In [228]:
import sys
import folium
import folium.plugins as fp
%matplotlib inline
import matplotlib.pyplot as plt

In [229]:
# split tags dict to columns and drop tags
intPlace = intPlace.join(intPlace['tags'].apply(pd.Series))
intPlace = intPlace.drop('tags', axis = 1)

In [230]:
# number of records for each tag column
nTags = intPlace.count().reset_index().sort_values(by = 0, ascending = False)
nTags.columns = ['amenity','count_tags']
# nTags.to_csv('nTags.csv')

In [231]:
# found canada place
intPlace[intPlace['addr:street'] == 'Canada Place'][['amenity','name']]

,amenity,name
1659,conference_centre,Vancouver Convention Centre East
2885,pub,Tap & Barrel
9084,restaurant,Botanist
10743,restaurant,Mahony and Sons
15921,cafe,Starbucks


In [232]:
# fill nan name with alt_name and drop alt_name
intPlace['name'] = intPlace['name'].fillna(intPlace['alt_name'])
intPlace['name'] = intPlace['name'].fillna(intPlace['official_name'])
# only keep columns with more than 100 records
columnKeep = list(nTags[nTags['count_tags'] > 1000]['amenity'])
intPlace = intPlace[columnKeep]

placeData = intPlace[intPlace['amenity'].isin(place)]

In [234]:
visitData = pd.read_csv('HLLBC_ATT.csv')
# trim semi colon:
def trim_semi(st):
    st = st.replace(' ;','')
    return(st)
visitData['DESCRIPTN'] = visitData['DESCRIPTN'].apply(trim_semi)
visitData = visitData[['DESCRIPTN','NAME','LATITUDE','LONGITUDE']]
visitKeep = ['Historic & Heritage Sites','Wineries','Wildlife Viewing - Whale Watching',
'Hiking - Hiking', 'Ziplining & Bungee Jumping','Kayaking & Canoeing - River',
'Kayaking & Canoeing - Ocean','Museums','Art Galleries', 'Parks - Provincial',
'Amusement Parks','Museums - Aboriginal Museums','Kayaking & Canoeing - Lake',
'Water Parks & Slides','Gardens & City Parks','Bridges, Buildings & Structures',
'Wildlife Viewing - Bird Watching', 'Shops & Malls','Science & Nature',
'Parks - National','Skiing & Snowboarding - Heli Skiing', 'Airtrams & Gondolas',
'Observatories & Planetariums','Aquariums','Historical & Heritage Sites - Aboriginal Sites','Hot Springs']
visitData = visitData[visitData['DESCRIPTN'].isin(visitKeep)]
visitData.columns = ['amenity','name','lat','lon']

placeData = pd.concat([placeData, visitData], sort=False)

In [ ]:
parks= pd.read_json('parks.json')
# split tags dict to columns and drop tags
parks = parks.join(parks['fields'].apply(pd.Series))
parks = parks.drop('fields', axis = 1)
# split tags dict to columns and drop tags
parks = parks.join(parks['geometry'].apply(pd.Series))
parks = parks.drop('geometry', axis = 1)
parks = parks[['datasetid','name','googlemapdest','hectare']]
googlemapdest = pd.DataFrame(parks['googlemapdest'].tolist(), columns=['lat','lon'])
parks= parks.join(googlemapdest)
parks = parks.drop('googlemapdest', axis = 1)
# parks.columns = ['amenity','name','lat','lon']

In [ ]:
# join parks and cults to intPlace
parks['amenity'] = 'park'
intPlace = pd.concat([intPlace, parks], sort=False)

In [43]:
# show option
def toSee(if_food = False, if_drink = False,if_place = False):
    toC = []
    if if_food == True:
        toC = toC+food
    if if_drink == True:
        toC = toC+drink
    if if_place == True:
        toC = toC+place
    return(toC)

In [ ]:
use model to get the points the visualize the point with arcgis. given multiple coordinates

In [237]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import geopandas as gpd

In [238]:
data_st = {'amenity':  ['skytrain', 'skytrain','skytrain', 'skytrain'],
        'name': ['Burrard', 'Granville','Waterfront','Stadium'],
        'lat':[49.2853095,49.2832795,49.2859559,49.279595],
        'lon':[-123.1221708,-123.1183089,-123.1137529,-123.1119618]
        }

stations = pd.DataFrame(data_st, columns = ['amenity','name','lat','lon'])

In [239]:
# adatped from https://towardsdatascience.com/nearest-neighbour-analysis-with-geospatial-data-7bcd95f34c0e
def create_gdf(df, x='lon', y='lat'):
    return gpd.GeoDataFrame(df, 
    geometry=gpd.points_from_xy(df[y], df[x]), 
             crs={'init':'EPSG:4326'})
stations_gdf = create_gdf(stations)
placeData_gdf = create_gdf(placeData)

In [240]:
def calculate_nearest(row, destination, val, col='geometry'):
    # 1 - create unary union    
    dest_unary = destination['geometry'].unary_union
    # 2 - find closest point
    nearest_geom = nearest_points(row[col], dest_unary)
    # 3 - Find the corresponding geom
    match_geom = destination.loc[destination.geometry == nearest_geom[1]]
    # 4 - get the corresponding value
    match_value = match_geom[val].values[0]
    return match_value

In [241]:
def df_nearest(station,data):
    station['nearest_geom'] = station.apply(calculate_nearest, destination=data, val='geometry', axis=1)
    station['nearest_place'] = station.apply(calculate_nearest, destination=data, val='name', axis=1)
    station['nearest_amenity'] = station.apply(calculate_nearest, destination=data, val='amenity', axis=1)
    station['nearest_lat'] = station.apply(calculate_nearest, destination=data, val='lat', axis=1)
    station['nearest_lon'] = station.apply(calculate_nearest, destination=data, val='lon', axis=1)
    station = station[['nearest_amenity','nearest_place','nearest_geom','nearest_lat','nearest_lon']]
    station.columns = ['amenity','name','geometry','lat','lon']
    return(station)
    
def get_route(station,data = placeData_gdf):
    station_data = stations_gdf[stations_gdf['name'] == station]
    result = pd.DataFrame(columns=['amenity','name','geometry','lat','lon'])

    nearest = df_nearest(station_data,data)
    result = result.append(nearest)
    data = data[data['name']!= nearest['name'].values[0]]
    
    nearest = df_nearest(nearest,data)
    result = result.append(nearest)
    data = data[data['name']!= nearest['name'].values[0] ]
    
    nearest = df_nearest(nearest,data)
    result = result.append(nearest)
    data = data[data['name']!= nearest['name'].values[0] ]
    
    nearest = df_nearest(nearest,data)
    result = result.append(nearest)
    data = data[data['name']!= nearest['name'].values[0] ]
    
    nearest = df_nearest(nearest,data)
    result = result.append(nearest)
#     data = data[data['name']!= nearest['name'].values[0] & data['amenity']!=nearest['amenity'].values[0]]
    return(result)
    
br = get_route(station = 'Burrard',data = placeData_gdf)
gv = get_route(station = 'Granville',data = placeData_gdf)
wf = get_route(station = 'Waterfront',data = placeData_gdf)
sd = get_route(station = 'Stadium',data = placeData_gdf)

#                          & data['amenity']!=nearest['nearest_amenity'].values[0]]

C:\Users\lehuy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\lehuy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lehuy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

C:\Users\lehuy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\lehuy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lehuy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [250]:
placeList = gv['name'].values.tolist()
placeDataPlot = gv[['lat','lon']]
locationlist = placeDataPlot.values.tolist()

map =folium.Map(location=[49.282730, -123.120735], zoom_start=15)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup = placeList[point]).add_to(map)

map

In [256]:
placeList = wf['name'].values.tolist()
placeDataPlot = wf[['lat','lon']]
locationlist = placeDataPlot.values.tolist()

map =folium.Map(location=[49.2859559,-123.1137529], zoom_start=15)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup = placeList[point]).add_to(map)

map

In [259]:
placeList = sd['name'].values.tolist()
placeDataPlot = sd[['lat','lon']]
locationlist = placeDataPlot.values.tolist()

map =folium.Map(location=[49.279595,-123.1119618], zoom_start=15)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup = placeList[point]).add_to(map)

map